# Goal: quantify the animal's performance in the W track

- to do that, we need access to the DIO
    - input: reward well
    - output: pumps
- we have extract DIO information rec file
    - Download Trodes (same version as one used for acquisition)
    - `trodesexport -rec FILENAME.rec -dio -time`
- identify which Din and Dout ports carry information we want
    - Din 1 and Dout 4 (Left)
    - Din 2 and Dout 5 (Center)
    - Din 7 and Dout 6 (Right)
- use python functions from SpikeGadgets to read the extracted binary files for DIO

## Function to read data extracted from rec files (from SpikeGadgets)

In [1]:
import numpy as np
import re
from sys import argv
# Main function
def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                line = line.decode('ascii').strip()
                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText


# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)


Example of how to open a digital input pin

In [4]:
Din2 = readTrodesExtractedDataFile('/stelmo/kyu/L5/20230502/r2/20230502_163824/20230502_163824.DIO/20230502_163824.dio_ECU_Din2.dat')

/tmp/ipykernel_200884/4104368914.py:61: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


In [9]:
Din2

{'description': 'State change data for one digital channel. Display_order is 1-based',
 'byte_order': 'little endian',
 'original_file': '20230502_163824.rec',
 'clockrate': '30000',
 'trodes_version': '2.3.4',
 'compile_date': 'Nov 28 2022',
 'compile_time': '16:20:03',
 'qt_version': '6.2.2',
 'commit_tag': 'heads/Release_2.3.3-0-g1975656c',
 'controller_firmware': '3.18',
 'headstage_firmware': '4.4',
 'controller_serialnum': '65535 65535',
 'headstage_serialnum': '01504 00126',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '1',
 'accelerometer': '1',
 'magnetometer': '0',
 'time_offset': '0',
 'system_time_at_creation': '1683070712537',
 'timestamp_at_creation': '32900161',
 'first_timestamp': '32949248',
 'direction': 'input',
 'id': 'ECU_Din2',
 'display_order': '2',
 'fields': '<time uint32><state uint8>',
 'data': array([(32949248, 1), (33054497, 0), (33055333, 1), (33145233, 0),
        (33146256, 1), (33154901, 0), (34017321, 1), (34021445, 0),
        (34022556, 1), (340227

Example of how to access timestamps
- there are two timestamps, Trodes and PTP
- use PTP (`systime`)

In [5]:
time = readTrodesExtractedDataFile('/stelmo/kyu/L5/20230502/r2/20230502_163824/20230502_163824.time/20230502_163824.timestamps.dat')

/tmp/ipykernel_200884/4104368914.py:61: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


In [6]:
time

{'description': 'Timestamps',
 'byte_order': 'little endian',
 'original_file': '20230502_163824.rec',
 'clockrate': '30000',
 'trodes_version': '2.3.4',
 'compile_date': 'Nov 28 2022',
 'compile_time': '16:20:03',
 'qt_version': '6.2.2',
 'commit_tag': 'heads/Release_2.3.3-0-g1975656c',
 'controller_firmware': '3.18',
 'headstage_firmware': '4.4',
 'controller_serialnum': '65535 65535',
 'headstage_serialnum': '01504 00126',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '1',
 'accelerometer': '1',
 'magnetometer': '0',
 'time_offset': '0',
 'system_time_at_creation': '1683070712537',
 'timestamp_at_creation': '32900161',
 'first_timestamp': '32949248',
 'decimation': '20',
 'fields': '<time uint32><systime int64>',
 'data': array([(32949248, 1683070714173554691), (32949249, 1683070714173563294),
        (32949250, 1683070714173602897), ...,
        (69741515, 1683071940590361006), (69741516, 1683071940590369211),
        (69741517, 1683071940590407421)],
       dtype=[('time', '<u4')